# External Data West Ameria(USA)

## Idaho

EO: 
- https://apps.sentinel-hub.com/eo-browser/?zoom=15&lat=46.43633&lng=-116.3649&themeId=DEFAULT-THEME
- https://apps.sentinel-hub.com/eo-browser/?zoom=14&lat=46.43605&lng=-116.35737&themeId=D[…]Z&toTime=2022-07-20T23%3A59%3A59.999Z&layerId=1_TRUE_COLOR


In [1]:
import os
import warnings
import random
from pprint import pprint
import copy
from typing import List, Tuple
import glob
import json
import csv
# import dataclasses
from joblib import Parallel, delayed
from typing import List, Set, Dict, Any

from tqdm import tqdm
import numpy as np
import pandas as pd
from box import Box, from_file
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from scipy.optimize import minimize
from sklearn.metrics import jaccard_score
import cv2
import seaborn as sns

In [24]:
class CFG(object):
    # basic
    debug: bool = False
    folds: int  = 4
    seed: int   = 417
    eps: float  = 1e-12
    
    # data
    PATH_CSV: str  =  f'../data/028_Idaho/019_seed{seed}.csv'
    PATH_NPY_ROOT = f'../data/028_Idaho/'
    PATH_DATA = '../data/Idaho/'
    os.makedirs(PATH_DATA, exist_ok=True), os.makedirs(PATH_NPY_ROOT + 'img/', exist_ok=True)
    
# box
cfg = Box({k:v for k, v in dict(vars(CFG)).items() if '__' not in k})
    
# 乱数のシードを設定
np.random.seed(cfg.seed)
random.seed(cfg.seed)
    
pprint(cfg)

{'PATH_CSV': '../data/028_Idaho/019_seed417.csv',
 'PATH_DATA': '../data/Idaho/',
 'PATH_NPY_ROOT': '../data/028_Idaho/',
 'debug': False,
 'eps': 1e-12,
 'folds': 4,
 'seed': 417}


## CLI

In [3]:
!aws --version

aws-cli/1.22.55 Python/3.8.0 Linux/5.4.0-150-generic botocore/1.24.0


## Download
few cloud scene

In [18]:
PATHS_SENTILEL2_S3 = [
    # # snow mountain area 
    # # Idaho
    "s3://sentinel-s2-l2a/tiles/11/T/NM/2023/3/7/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/NM/2023/2/25/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/NM/2022/10/18/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/NM/2022/10/8/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/NM/2022/10/3/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/NM/2022/7/15/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/NM/2022/2/25/0/",
    "s3://sentinel-s2-l2a/tiles/11/T/NM/2022/1/26/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2022/1/16/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2021/8/29/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2021/8/4/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2021/6/25/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2021/5/16/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2021/3/17/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2021/3/12/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2020/12/2/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2020/9/28/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2020/7/5/0/",
    # "s3://sentinel-s2-l2a/tiles/11/T/NM/2020/6/25/0/"    
]

In [5]:
for PATH_S3 in tqdm(PATHS_SENTILEL2_S3):
    SCENE_ID = PATH_S3.split('tiles')[1].replace('/', '-')[1:-3]
    print(SCENE_ID)
    OUTPUT_PATH = f'{cfg.PATH_DATA}/{SCENE_ID}/'
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    
    !aws s3 cp {PATH_S3} {OUTPUT_PATH} --recursive --request-payer
    # break

  0%|          | 0/20 [00:00<?, ?it/s]

11-T-NM-2023-3-7
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/3/7/0/R10m/AOT.jp2 to ../data/Oregon/11-T-NM-2023-3-7/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/3/7/0/R10m/B08.jp2 to ../data/Oregon/11-T-NM-2023-3-7/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/3/7/0/R10m/TCI.jp2 to ../data/Oregon/11-T-NM-2023-3-7/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/3/7/0/R10m/B03.jp2 to ../data/Oregon/11-T-NM-2023-3-7/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/3/7/0/R10m/B02.jp2 to ../data/Oregon/11-T-NM-2023-3-7/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/3/7/0/R20m/AOT.jp2 to ../data/Oregon/11-T-NM-2023-3-7/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/3/7/0/R10m/WVP.jp2 to ../data/Oregon/11-T-NM-2023-3-7/R10m/WVP.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/3/7/0/R10m/B04.jp2 to ../data/Oregon/11-T-NM-2023-3-7/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/20

  5%|▌         | 1/20 [04:31<1:25:56, 271.37s/it]

11-T-NM-2023-2-25
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/2/25/0/R10m/AOT.jp2 to ../data/Oregon/11-T-NM-2023-2-25/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/2/25/0/R10m/B02.jp2 to ../data/Oregon/11-T-NM-2023-2-25/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/2/25/0/R10m/B03.jp2 to ../data/Oregon/11-T-NM-2023-2-25/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/2/25/0/R20m/AOT.jp2 to ../data/Oregon/11-T-NM-2023-2-25/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/2/25/0/R10m/B04.jp2 to ../data/Oregon/11-T-NM-2023-2-25/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/2/25/0/R10m/B08.jp2 to ../data/Oregon/11-T-NM-2023-2-25/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/2/25/0/R10m/WVP.jp2 to ../data/Oregon/11-T-NM-2023-2-25/R10m/WVP.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2023/2/25/0/R20m/B03.jp2 to ../data/Oregon/11-T-NM-2023-2-25/R20m/B03.jp2
download: s3://sentinel-s2-l2a

 10%|█         | 2/20 [09:01<1:21:09, 270.50s/it]

...

 90%|█████████ | 18/20 [1:13:10<07:41, 230.56s/it]

11-T-NM-2020-7-5
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/7/5/0/R10m/AOT.jp2 to ../data/Oregon/11-T-NM-2020-7-5/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/7/5/0/R10m/B04.jp2 to ../data/Oregon/11-T-NM-2020-7-5/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/7/5/0/R10m/B03.jp2 to ../data/Oregon/11-T-NM-2020-7-5/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/7/5/0/R20m/AOT.jp2 to ../data/Oregon/11-T-NM-2020-7-5/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/7/5/0/R10m/B02.jp2 to ../data/Oregon/11-T-NM-2020-7-5/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/7/5/0/R10m/TCI.jp2 to ../data/Oregon/11-T-NM-2020-7-5/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/7/5/0/R20m/B02.jp2 to ../data/Oregon/11-T-NM-2020-7-5/R20m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/7/5/0/R10m/B08.jp2 to ../data/Oregon/11-T-NM-2020-7-5/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/20

 95%|█████████▌| 19/20 [1:17:09<03:53, 233.06s/it]

11-T-NM-2020-6-25
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/6/25/0/R10m/AOT.jp2 to ../data/Oregon/11-T-NM-2020-6-25/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/6/25/0/R10m/B02.jp2 to ../data/Oregon/11-T-NM-2020-6-25/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/6/25/0/R10m/B04.jp2 to ../data/Oregon/11-T-NM-2020-6-25/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/6/25/0/R20m/AOT.jp2 to ../data/Oregon/11-T-NM-2020-6-25/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/6/25/0/R10m/B03.jp2 to ../data/Oregon/11-T-NM-2020-6-25/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/6/25/0/R10m/TCI.jp2 to ../data/Oregon/11-T-NM-2020-6-25/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/6/25/0/R10m/B08.jp2 to ../data/Oregon/11-T-NM-2020-6-25/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/11/T/NM/2020/6/25/0/R20m/B02.jp2 to ../data/Oregon/11-T-NM-2020-6-25/R20m/B02.jp2
download: s3://sentinel-s2-l2a

100%|██████████| 20/20 [1:20:42<00:00, 242.12s/it]


## Loading data

In [6]:
BANDS = [
    'R20m/B01', 'R10m/B02', 'R10m/B03', 'R10m/B04', 'R20m/B05', 'R20m/B06', 
    'R20m/B07', 'R10m/B08', 'R20m/B8A', 'R60m/B09', 'R20m/B11', 'R20m/B12',
]

In [19]:
PATHS_NPY = []

for PATH_S3 in tqdm(PATHS_SENTILEL2_S3):
    SCENE_ID = PATH_S3.split('tiles')[1].replace('/', '-')[1:-3]
    print(SCENE_ID)
    OUTPUT_PATH = f'{cfg.PATH_DATA}/{SCENE_ID}/'
    
    imgs = []
    
    for BAND in BANDS:
        PATH_BAND = f'{OUTPUT_PATH}{BAND}.jp2'
        img = cv2.imread(PATH_BAND, cv2.IMREAD_UNCHANGED)        
        if 'R20m' in BAND:
            img = cv2.resize(img, (img.shape[1]*2, img.shape[0]*2), interpolation=1)
        elif 'R60m' in BAND:
            img = cv2.resize(img, (img.shape[1]*6, img.shape[0]*6), interpolation=1)

        imgs.append(img)
        
    imgs = np.stack(imgs, axis=2)
    
    for h_idx in range(imgs.shape[0]//512):
        for w_idx in range(imgs.shape[1]//512):
            
            patch = imgs[h_idx:h_idx+512, w_idx:w_idx+512, :]
                
            PATH_NPY = f'{cfg.PATH_NPY_ROOT}img/{SCENE_ID}_h{h_idx}_w{w_idx}.npy'
            np.save(PATH_NPY, patch)
            PATHS_NPY.append(PATH_NPY)


  0%|          | 0/8 [00:00<?, ?it/s]

11-T-NM-2023-3-7


 12%|█▎        | 1/8 [02:12<15:25, 132.20s/it]

11-T-NM-2023-2-25


 25%|██▌       | 2/8 [04:27<13:22, 133.75s/it]

...

 75%|███████▌  | 6/8 [13:01<04:18, 129.15s/it]

11-T-NM-2022-2-25


 88%|████████▊ | 7/8 [15:16<02:11, 131.03s/it]

11-T-NM-2022-1-26


100%|██████████| 8/8 [17:31<00:00, 131.39s/it]


In [20]:
df = pd.DataFrame({
    'feature': PATHS_NPY
})

df['name'] = df['feature'].apply(lambda x: x.split('/')[-1].split('.')[0])
df['mask'] = False
df['defects'] = False
df['fold'] = -1
df['hight'] = 512
df['width'] = 512
df['california'] = False
df['cv'] = -1
df['area'] = 'Idaho'

df.head(4)

,feature,name,mask,defects,fold,hight,width,california,cv,area
0,../data/028_Idaho/img/11-T-NM-2023-3-7_h0_w0.npy,11-T-NM-2023-3-7_h0_w0,False,False,-1,512,512,False,-1,Idaho
1,../data/028_Idaho/img/11-T-NM-2023-3-7_h0_w1.npy,11-T-NM-2023-3-7_h0_w1,False,False,-1,512,512,False,-1,Idaho
2,../data/028_Idaho/img/11-T-NM-2023-3-7_h0_w2.npy,11-T-NM-2023-3-7_h0_w2,False,False,-1,512,512,False,-1,Idaho
3,../data/028_Idaho/img/11-T-NM-2023-3-7_h0_w3.npy,11-T-NM-2023-3-7_h0_w3,False,False,-1,512,512,False,-1,Idaho


In [25]:
df.to_csv(cfg.PATH_CSV, index=False, header=True)